In [5]:
import torch
import torch.nn as nn
from torchsummary import summary
import torchvision
import hiddenlayer
import matplotlib.pyplot as plt

import sys

sys.path.append('/Users/mateo/Desktop/FER/SEM4/DIPL/PROJECT_pytorch')

import models

device = torch.device('cpu')

BATCH_SIZE = 64
INPUT_DIM = 1000
HIDDEN_DIM = 100
OUTPUT_DIM = 10

x = torch.randn(BATCH_SIZE, INPUT_DIM, device=device)
y = torch.randn(BATCH_SIZE, OUTPUT_DIM, device=device)

# model = nn.Sequential(
#     nn.Linear(INPUT_DIM, HIDDEN_DIM),
#     nn.ReLU(),
#     nn.Linear(HIDDEN_DIM, OUTPUT_DIM)
# ).to(device)
#
# loss_fn = nn.MSELoss(reduction='sum')
# lr = 1e-4
#
# for i in range(500):
#     y_pred = model(x)
#
#     loss = loss_fn(y_pred, y)
#     print(i, loss.item())
#
#     # COMMENT set gradients to zero
#     model.zero_grad()
#
#     # COMMENT backpropagation
#     loss.backward()
#
#     with torch.no_grad():
#         for param in model.parameters():
#             param.data -= lr * param.grad
#
# summary(model, input_size=(1, INPUT_DIM))

model = models.HourglassModule(
    num_blocks=99,
    planes=99,
    depth=4
).to(device)

summary(model, (256, 64, 64))

hiddenlayer.build_graph(model, torch.zeros([8, 256, 64, 64]))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 64, 64]          32,896
       BatchNorm2d-2          [-1, 128, 64, 64]             256
              ReLU-3          [-1, 128, 64, 64]               0
            Conv2d-4          [-1, 128, 64, 64]         147,584
       BatchNorm2d-5          [-1, 128, 64, 64]             256
              ReLU-6          [-1, 128, 64, 64]               0
            Conv2d-7          [-1, 256, 64, 64]          33,024
       BatchNorm2d-8          [-1, 256, 64, 64]             512
              ReLU-9          [-1, 256, 64, 64]               0
   ResidualModule-10          [-1, 256, 64, 64]               0
           Conv2d-11          [-1, 128, 32, 32]          32,896
      BatchNorm2d-12          [-1, 128, 32, 32]             256
             ReLU-13          [-1, 128, 32, 32]               0
           Conv2d-14          [-1, 128,

ModuleNotFoundError: No module named 'graphviz'